In [28]:
import requests
import json
import datetime as dt
from hdfs import InsecureClient
client = InsecureClient('http://localhost:9870', user='big') #나의 로컬호스트주소 #user : big(내가정함)

In [9]:
# 라이브러리 입장에서는 404도 응답이기 때문에 에러가 아니다. 
# 우리가 그런상황도 error라고 정해 놓고 가된다.

# raise Exception('응답코드 : ' + str(404))  의도적 에러 Exception: 응답코드 : 404

def execute_rest_api(method, url, headers, params):
    if method == 'get':
        res = requests.get(url,params=params,headers=headers)
    elif method == 'post' :
        res = requests.post(url,params=params,headers=headers)
    
    if res ==None or res.status_code != 200 :
        raise Exception('응답코드 : ' + str(res.status_code))
        
    return res.text

In [3]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [4]:
# 로그만들기
import logging
# 내가원하는 과정중에 발생한 에러만 골라서 기록하는 로깅이 필요
co_logger = logging.getLogger('corona_api')
# 로그를 한파일에만 넣으면 나중에 찾기 힘들고 심지어 열기조차 불가능 할 수 있음 -> 이번엔 날짜별로 하자, 하둡은 파일 크기별로함
handler = logging.FileHandler('./log/rest_api/'+cal_std_day(0)+'.log')
co_logger.addHandler(handler)

In [41]:
##식품의약품안전처_나트륨 줄이기 실천음식점 지정업체 대장 api 크롤링
url = 'http://apis.data.go.kr/1471000/NaReduceRstrtDsgnBsshRgstr/getNaReduceRstrtDsgnBsshRgstr'
service_key = '7PVnzAEB7lyY+l/eu/AFIJeUgExZ++Lg1F7gaQNgVsUAQn0+sw5q87JxBSJKLl95ASpOdP+CnF7MOBh8fMcJVQ=='
file_dir =  '/eat/'
find_name = '서울특별시 '
# 서울시 자치구 25개
seoul_gu = [
            '강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구','도봉구','동대문구','동작구','마포구','서대문구',
           '서초구','성동구','성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구'
           ]
# len(seoul_gu) # 25


def create_param(gu_num):
    return {
        'serviceKey':service_key,
        'pageNo':'1',
        'numOfRows':'100',
        'type':'JSON',
    }
# for i in range(len(seoul_gu))
#     #파라미터 만들기 서울행정구 이름으로
#     try :
#         params = create_param(i)
#         res = execute_rest_api('get',url,{},params)
#         file_name = 'healthy_restaurant' + params[cal_std_day(1)] + '.json'
#         client.write(file_dir+file_name,res,encoding='utf-8'
#     except Exception as e :
#         # 로그파일이 있어야만 안에 로그가 기록이됨 #선이 로그파일 생성
#         log_dict['err_msg']= e.__str__()
#         log_json = json.dumps(log_dict,ensure_ascii=False)
#         co_logger.error(log_json)


In [59]:
params = create_param(0)
res = execute_rest_api('get',url,{},params)
file_name = 'healthy_restaurant_' + cal_std_day(1) + '.csv'
client.write(file_dir+file_name,res,encoding='utf-8')

In [61]:
path = '/eat/healthy_restaurant_2022-09-26.json'
clean_res = spark.read.csv(path,encoding='UTF-8')
clean_res.show()

+---------------------------+
|                        _c0|
+---------------------------+
|       <?xml version="1....|
|                 <response>|
|                   <header>|
|       <resultCode>00</r...|
|       <resultMsg>NORMAL...|
|                  </header>|
|                     <body>|
|       <numOfRows>100</n...|
|         <pageNo>1</pageNo>|
|       <totalCount>2146<...|
|                    <items>|
|                     <item>|
|       <LCNS_NO>20090121...|
|<BSSH_NM>노랑통닭 명일점...|
|    <PRSDNT_NM>김*준 외 ...|
|    <INDUTY_CD_NM>일반음...|
|  <SITE_ADDR>서울특별시 ...|
|       <PART_GBN_DIV_NM>...|
|                <JOIN_YMD/>|
|                <APPT_YMD/>|
+---------------------------+
only showing top 20 rows



In [70]:
raw_data = clean_res.collect()[10:]
raw_data

[Row(_c0='<items>'),
 Row(_c0='<item>'),
 Row(_c0='<LCNS_NO>20090121006</LCNS_NO>'),
 Row(_c0='<BSSH_NM>노랑통닭 명일점</BSSH_NM>'),
 Row(_c0='<PRSDNT_NM>김*준 외 1명</PRSDNT_NM>'),
 Row(_c0='<INDUTY_CD_NM>일반음식점</INDUTY_CD_NM>'),
 Row(_c0='<SITE_ADDR>서울특별시 강동구 구천면로 442(1층 명일동)</SITE_ADDR>'),
 Row(_c0='<PART_GBN_DIV_NM>지정</PART_GBN_DIV_NM>'),
 Row(_c0='<JOIN_YMD/>'),
 Row(_c0='<APPT_YMD/>'),
 Row(_c0='<ETC_INFO/>'),
 Row(_c0='<INSTT_NM>서울특별시 강동구</INSTT_NM>'),
 Row(_c0='</item>'),
 Row(_c0='<item>'),
 Row(_c0='<LCNS_NO>20050443130</LCNS_NO>'),
 Row(_c0='<BSSH_NM>곰가내</BSSH_NM>'),
 Row(_c0='<PRSDNT_NM>안*희</PRSDNT_NM>'),
 Row(_c0='<INDUTY_CD_NM>일반음식점</INDUTY_CD_NM>'),
 Row(_c0='<SITE_ADDR>충청북도 진천군 백곡면 백곡로 835-8</SITE_ADDR>'),
 Row(_c0='<PART_GBN_DIV_NM>지정</PART_GBN_DIV_NM>'),
 Row(_c0='<JOIN_YMD/>'),
 Row(_c0='<APPT_YMD/>'),
 Row(_c0='<ETC_INFO/>'),
 Row(_c0='<INSTT_NM>충청북도 진천군</INSTT_NM>'),
 Row(_c0='</item>'),
 Row(_c0='<item>'),
 Row(_c0='<LCNS_NO>20150380254</LCNS_NO>'),
 Row(_c0='<BSSH_NM>섬마을</BS

In [102]:
name_list = []
addr_list = []
code_list = []
sigungu_list = []

for i in range(11,len(raw_data),12 ):
    if "서울" in raw_data[i][0][10:][:-11]:
        sigungu_list.append(raw_data[i][0][10:][:-11])
        addr_list.append(raw_data[i-5][0][11:][:-12])
        code_list.append(raw_data[i-6][0][14:][:-15])
        name_list.append(raw_data[i-8][0][9:][:-10])


                

In [104]:
sigungu_list
name_list

['노랑통닭 명일점',
 '노랑통닭 한양대점',
 '노랑통닭 대학로점',
 '노랑통닭 노원역점',
 '노랑통닭 고척점',
 '노랑통닭 영등포구청역점',
 '노랑통닭',
 '노랑통닭',
 '노랑통닭 아현점',
 '고피자 다트플렉스 이수점',
 '노랑통닭 경희대점']